In [9]:
import io
import numpy as np
import tensorflow as tf

tf.enable_eager_execution()

In [2]:
filename = '/tmp/data/mnist/tmpbjd0WU/train_shuffled-00000-of-00001.gz'

In [3]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames, compression_type='GZIP')

In [4]:
FEATURE_DESCRIPTION = {
    'image': tf.FixedLenFeature([], tf.string, default_value=''),
    'label': tf.FixedLenFeature([], tf.int64, default_value=0),
}

def _parse_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    return tf.parse_single_example(example_proto, FEATURE_DESCRIPTION)

parsed_dataset = raw_dataset.map(_parse_function)

In [7]:
record = next(iter(parsed_dataset.take(1)))
type(record), record

(dict,
 {'image': <tf.Tensor: id=54, shape=(), dtype=string, numpy="\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00C\xe8'\x00\x00\x00\x00\x00\x00\x00\x00\x00>Q\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00x\xb4'\x00\x00\x00\x00\x00\x00\x00\x00\x00~\xa3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x99\xd2(\x00\x00\x00\x00\x00\x00\x00\x00\x00\xdc\xa3\x00\x00\x00\x00\x00\x00\x00\x00\x00

### decode to `np.ndarray`

In [11]:
def convert_parsed_record_to_ndarray(parsed_record):
    x = parsed_record['image']
    x_np = x.numpy()
    bytestream = io.BytesIO(x_np)
    rows = 28
    cols = 28
    num_images = 1
    buf = bytestream.read(rows * cols * num_images)
    data = np.frombuffer(buf, dtype=np.uint8)
    shape = (rows, cols, num_images)
    data = data.reshape(*shape)
    assert isinstance(data, np.ndarray), type(data)
    assert data.shape == shape
    return data

img = convert_parsed_record_to_ndarray(record)
type(img), img.shape

(numpy.ndarray, (28, 28, 1))

In [21]:
record['label']

<tf.Tensor: id=55, shape=(), dtype=int64, numpy=4>

### convert byte string tensor to array

In [14]:
IMAGE_KEY = 'image'
LABEL_KEY = 'label'
IMAGE_PIXELS = 784
features = record

image = tf.decode_raw(features[IMAGE_KEY], tf.uint8)
image.set_shape((IMAGE_PIXELS))  # 784

# Convert label from a scalar uint8 tensor to an int32 scalar.
label = tf.cast(features[LABEL_KEY], tf.int32)

In [16]:
image.shape

TensorShape([Dimension(784)])

In [20]:
label

<tf.Tensor: id=63, shape=(), dtype=int32, numpy=4>

In [22]:
!pwd

/Users/aaron/Documents/github/transform
